# Creating seismic synthetics with Devito and GemPy
In this tutorial, we aim to provide a start-to-finish guide for creating synthetic seismic models and datasets using the open-source packages Devito and GemPy. We will highlight the steps involved, from creating a 3D geological model, to forward modelling, to visualisation.

## Goals
* Introduce users to creating simple geological models with GemPy
* Illustrate how to include physical properties in the geological model and how to convert this into a format usable for seismic modelling with Devito
* Set up a Devito `operator` to solve the 2nd order acoustic wave equation, including sparse sources and receivers
* Forward propagate a shot through the model, and record a synthetic gather using receivers
* Visualise the wavefield in 3D using slicing in PyVista

## Structure
This tutorial will be presented in three sections. The first section will focus on creating the seismic synthetic, the second on model setup and the Devito API, and the latter will cover running the seismic wave model and visualising the wavefield.

Throughout the tutorial, several members of the team will be available in the `#t21-thurs-devito` channel of the [Software Underground Slack](https://softwareunderground.org/slack). For in-depth GemPy questions, the `#gempy` channel is another option. For Devito-centric questions, try the `#devito` channel or the [Devito Slack](https://devitocodes.slack.com/).

## Motivation
Synthetic seismic models are crucial for testing and benchmarking numerous seismic modelling and imaging applications. Whilst several seismic synthetics  are commonplace within the community for example the ubiquitous Marmousi model, or the SEG/EAGE Salt and Overthrust Models, the range of freely-available synthetics is relatively small, with other purpose-built models carrying a steep cost. Furthermore currently-available seismic synthetics may not bear sufficient resemblence to the desired target, or may be lacking certain material properties required for a particular wave equation. This has implications for both accessibility and reproducibility of studies, as benchmarks used may not be readily available for others to verify the work.

By using freely-available tools to create synthetic models and datasets, we can create open, sharable, customisable models for a wide range of applications.

## Side Note (Google Colab only)

If running on Google Colab, you will need to run the following cells to prevent 3D renders crashing your notebook.

In [ ]:
"""
!apt-get update
!apt-get -qq install xvfb
!pip install pyvirtualdisplay
"""

In [ ]:
"""
from pyvirtualdisplay import Display
display = Display(visible=0, size=(600, 400))
display.start()
"""

## GemPy installation
![gempy_model.png](https://github.com/devitocodes/transform2021/blob/main/figures/gempy_model.png?raw=1)
[Source](https://docs.gempy.org/examples/real/Greenstone.html#sphx-glr-download-examples-real-greenstone-py): An example of a detailed geological model, taken from the Greenstone GemPy example.

The synthetics which we will be building in this tutorial will be made with the use of GemPy, an open-source 3D geological modelling package for Python. If not already installed, we will need to install it. If issues are encountered whilst installing GemPy into a `conda` environment using `pip`, you can alternatively create a python `venv` and install into this environment using `pip` as per usual. Note that it will also be necesary to install an `ipykernel` in this environment to run this notebook. From here, we can install GemPy:

In [ ]:
try:
    # Import gempy
    import gempy as gp
except ModuleNotFoundError:
    # Install gempy
    ! pip install gempy
    # Import gempy
    import gempy as gp

# Check jinja2 is installed for colour-coding of geological units
try:
    import jinja2
except ModuleNotFoundError:
    ! pip install jinja2
    import jinja2

## Model overview

The simple geological model which we will be building is designed to evoke carbon-capture and storage (CCS) scenarios.

The model consists of a CO2 lens in a sandstone reservoir, with a shale layer in the overarching anticline providing the structural trap. This is then overlain by a layer of sediment, with water at the top of the model. Geological strata and their respective velocities are based on values detailed in [Queißer et al. 2013](https://doi.org/10.1190/geo2012-0216.1), a paper imaging the P-wave velocity anomaly generated by CO2 injection into the Utsira Sand at Sleipnir in the North Sea using FWI. The model we will create features a similar shale trap/permeable sandstone reservoir structure, albeit with a small number of thick layers rather than the thin interbedding, to limit model complexity for this tutorial. Further inspiration was taken from [Chadwick et al. 2004](https://doi.org/10.1016/j.energy.2004.03.071), a paper characterizing the Utsira Sand reservoir based on 2D seismic lines and well logs.


## Creating our geological model:

To begin, alongside GemPy, we need to import some  auxiliary modules:

In [ ]:
# Import auxiliary modules
import numpy as np

%matplotlib inline

### Initial setup

We will now set up a GemPy `Model` object. This encapsulates the grid onto which the scalar fields associated with various surfaces are interpolated. Note that the extent is slightly greater than it will be for our Devito model (an extra half a grid spacing is added to each side).

![gempy_devito_grid_diagram.png](https://github.com/devitocodes/transform2021/blob/main/figures/gempy_devito_grid_diagram.png?raw=1)
A comparison of the cell-centered vs node-centered conventions of GemPy and Devito respectively, along with the differences in how they measure extent. It is necessary to account for this to ensure that the two grids are co-located.

As we can see in the figure above, this is due to differences in the way in which grids are defined in each package and is necessary to ensure that the model is not stretched and distorted when transistioning between the two, and that they are correctly aligned.

In [ ]:
# Set overarching model parameters
extent = (-5., 1005., -5., 1005., -1005., 5.)
shape = (101, 101, 101)

geo_model = gp.create_model('transform-2021')
geo_model = gp.init_data(geo_model, extent=extent, resolution=shape)

We now need to set up Theano for our model (used by GemPy for interpolation of model properties). Bear in mind that that this may take some time to run.

In [ ]:
gp.set_interpolator(geo_model, output=['geology'], theano_optimizer='fast_compile')

### Adding geological units
As the top CO2 surface is truncated by the upper shale layer, we will need to separate the geological strata into two GemPy `Series`. Each `Series` object, as the name suggests is intended to correspond with a geological unit, and they can be made to onlap, erode, etc one another. Whilst in practice, the top CO2 contact is not an erosive surface, treating it as such is the most straightforward way to create the desired truncation, as the shale trap is unconformable on the CO2 lens and reservoir sandstone.

A default series is included in the model. As such, rather than creating a new series, we will simply rename it to 'Lower'. As you can imagine, this is going to be used to contain the lower geological units, these being the lower shale, reservoir sandstone, and CO2 lens.

In [ ]:
geo_model.rename_series({'Default series': 'Lower'})

And now add our surfaces:

In [ ]:
geo_model.add_surfaces(['co2', 'sands', 'lowershale'])

We will now set some points for the base of the sands and CO2. The lower shale is considered the basement, meaning that its base does not need to be defined and it will extend to the bottom of the model. Alongside these points, we wil need to define an orientation for the surface.

To minimise repetition, we will define a function to loop over a list of points and add each to the surface.

In [ ]:
def create_surface(model, points, surface):
    """Add a list of points to a surface in a model"""
    xyz = ('X', 'Y', 'Z')
    for point in points:
        kwargs = {**dict(zip(xyz, point)), 'surface': surface}
        model.add_surface_points(**kwargs)

# The points defining the base of the sand layer
sand_points = [(322, 135, -783), (635, 702, -791), (221, 668, -772), (732, 235, -801), (442, 454, -702)]

# Call our function
create_surface(geo_model, sand_points, 'sands')

# Add the surface orientation
geo_model.add_orientations(X=442., Y=495., Z=-752.,
                           surface='sands', pole_vector=(0.05, 0.05, 0.95))

We will now repeat this process for the CO2 lens.

In [ ]:
# Points defining the base of the CO2 layer
co2_points = [(322, 135, -650), (635, 702, -650), (221, 668, -650), (732, 235, -650), (442, 454, -650)]

create_surface(geo_model, co2_points, 'co2')

# Add the surface orientation
geo_model.add_orientations(X=495., Y=495., Z=-650.,
                           surface='co2', pole_vector=(0., 0., 1.))

We will now add an upper series, containing statigraphy above the CO2 lens.

In [ ]:
geo_model.add_series('Upper')

As we can see, the upper series has been added below the lower series. This is not ideal for obvious reasons, and hence we will reorder them:

In [ ]:
geo_model.reorder_series(['Upper', 'Lower'])

And add our remaining surfaces:

In [ ]:
geo_model.add_surfaces(['water', 'sediments', 'uppershale'])

As these surfaces are not mapped to the upper series by default, we shall do so:

In [ ]:
gp.map_stack_to_surfaces(geo_model, {'Upper': ('water', 'sediments', 'uppershale')})

Now we will add the points for the upper series. Note that there is only a single orientation included. It is not necessary to define an orientation for each surface, so long as there is an orientation in the series.

In [ ]:
# Surface points
uppershale_points = [(322, 135, -633), (635, 702, -641), (221, 668, -622), (732, 235, -651), (442, 454, -552)]
sediments_points = [(322, 135, -433), (635, 702, -441), (221, 668, -422), (732, 235, -451), (442, 454, -352)]
water_points = [(232, 153, -221), (653, 234, -216), (112, 872, -198), (532, 572, -223),
                (722, 884, -189), (632, 429, -201), (732, 348, -222)]

# Add the points to our surfaces
create_surface(geo_model, uppershale_points, 'uppershale')
create_surface(geo_model, sediments_points, 'sediments')
create_surface(geo_model, water_points, 'water')

# Set an orientation
geo_model.add_orientations(X=442., Y=495., Z=-502.,
                           surface='uppershale', pole_vector=(0.05, 0.05, 0.95))

### Adding physical properties (Vp)

Finally, we can add the p-wave velocities associated with each of these layers. Note that any parameter can be set in this manner (density, elastic parameters, attenuation, etc) if desired for more complex synthetics. This is one of the benefits of using GemPy to create these synthetics: models can be expanded upon with a huge suite of material properties.

In [ ]:
geo_model.add_surface_values([1.5, 1.75, 2.5, 1.1, 2., 2.5], ['vp'])
geo_model.surfaces

### Visualising the GemPy model

Now we can visualise our model, plotting data points and orientations. Firstly, the model must be computed to interpolate the surfaces and any associated scalar fields. Then we can plot our surfaces and the associated units.

In [ ]:
# Set up plotter
p3d = gp.plot_3d(geo_model, notebook=True)
# Plot data points and orientations
p3d.plot_data()

# Compute the model. Note that a solution is returned. We will use this later
sol = gp.compute_model(geo_model)

# Plot the surfaces
p3d.plot_surfaces()
# Plot the lithological units
p3d.plot_structured_grid('lith')

## Bridging the gap from GemPy to Devito:

As you may have noticed, when we compute our GemPy model, a `Solution` object is returned. From this, we can extract the rasterized values attached to each of our geological units. With this in mind, we can print the solution values:

In [ ]:
sol.values_matrix

You will notice that these values correspond with the p-wave velocities we specified. However, they are in the form of 1D vector. Consequently, will need to reshape this array to fit into the `vp` parameter of a Devito `Model`. This can be done with further parameters such as density or shear wave velocity for more complex models. In this case, you would want to set up a Devito `Function` to contain each parameter.

Note that in this case, we need to select c-like index order to get the axes in the correct order.

In [ ]:
# Reshaping our data to the shape required by Devito
reshaped = np.reshape(sol.values_matrix, shape, order='C')
reshaped.shape

### Quality checking

Now let us plot a slice through this model for quality checking purposes. Note that we need to bear Devito's `[x, y, z]` indexing convention in mind.

In [ ]:
import matplotlib.pyplot as plt

# Take the center slice in the x direction
# Remember that in Devito, indexing convention is [x, y, z] (need to flip for correct imshow display)
plt.imshow(reshaped[50].T, cmap='viridis', origin='lower')
plt.colorbar()
plt.show()

The model looks good, and we can see the geometry of the CO2 lens clearly. We are now ready to begin our seismic modelling with Devito.

## Break Time!
Time to go and get a coffee.

## Seismic modelling with Devito

We can now start building our Devito model. The following draws heavily from the Devito `examples/sesimic/tutorials/01_modelling.ipynb` notebook (see [here](https://github.com/devitocodes/devito/blob/master/examples/seismic/tutorials/01_modelling.ipynb)). We will begin, as always with some imports. If Devito is not installed, we will need to install it.

We will also be drawing on some convenient helper functions for "toy" seismic models from `examples.seismic`. Whilst these are not part of core Devito and probably shouldn't be used in practical implementations, they are helpful for the purposes of learning the workflow.

In [ ]:
try:
    # Import devito
    import devito as dv
    from examples.seismic import Model
except ModuleNotFoundError:
    # Install newest SymPy version
    ! pip install -Iv sympy==1.8
    # Install devito
    ! pip install devito
    # Import devito
    import devito as dv
    from examples.seismic import Model

### Modelling workflow
The core process we are aiming to model is a seismic survey, which consists of two main components:
* **Source** - A source is positioned at a single or a few physical locations where artificial pressure is injected into the domain we want to model. In the case of land survey, it is usually dynamite, or a vibroseis (a truck fitted with a vibrating plate generating continuous sound waves). For a marine survey, the source is an air gun sending a bubble of compressed air into the water that will expand and generate a seismic wave.
* **Receivers** - A set of geophones or hydrophones are used to measure the resulting wave and create a set of measurements called a "Shot Record" or "Shot Gather". These measurements are recorded across an array, usually located near the surface, although they may be a depth in the case of ocean-bottom seismometers (OBS) or downhole receivers.

In order to create a numerical model of a seismic survey, we need to solve a wave equation and implement source and receiver interpolation to inject the source and record the seismic wave at sparse point locations in the grid.

![survey-ship-diagram.png](https://github.com/devitocodes/transform2021/blob/main/figures/survey-ship-diagram.png?raw=1)
[Source](https://upload.wikimedia.org/wikipedia/commons/0/01/Diagram_of_a_marine_seismic_survey.png) A diagram of a marine seismic survey.

### The acoustic wave equation
The acoustic wave equation for the square slowness $m$, defined as $m=\frac{1}{c^2}$, where $c$ is the speed of sound in the given physical media, and a source $q$ is given by:

\begin{cases}
 &m \frac{d^2 u(x,t)}{dt^2} - \nabla^2 u(x,t) = q \ \text{in } \Omega \\
 &u(.,t=0) = 0 \\
 &\frac{d u(x,t)}{dt}|_{t=0} = 0 
\end{cases}

with the zero initial conditions to guarantee unicity of the solution.
The boundary conditions are Dirichlet conditions:
\begin{equation}
 u(x,t)|_\delta\Omega = 0
\end{equation}

where $\delta\Omega$ is the surface of the boundary of the model $\Omega$.

### Finite domains

The last piece of the puzzle is the computational limitation. In the field, the seismic wave propagates in every direction to an "infinite" distance. However, solving the wave equation in a mathematically/discrete infinite domain is not feasible. In order to compensate, Absorbing Boundary Conditions (ABC) or Perfectly Matched Layers (PML) are required to mimic an infinite domain. These two methods allow to approximate an infinite media by damping and absorbing the waves at the limit of the domain to avoid reflections.

The simplest of these methods is the absorbing boundary region. The core idea is to extend the physical domain and to add an absorbing region in this extension that will absorb the incident waves. The acoustic wave equation with this damping region can be rewritten as:

\begin{cases} 
 &m \frac{\partial^2 u(\textbf{x},t)}{\partial t^2} - \nabla^2 u(\textbf{x},t) + \eta \frac{\partial u(\textbf{x},t)}{\partial t}=q  \ \text{in } \Omega \\
 &u(.,0) = 0 \\
 &\frac{\partial u(\textbf{x},t)}{\partial t}|_{t=0} = 0 
\end{cases}

where $\eta$ is the damping coefficient equal to $0$ inside the physical domain and increasing inside the absorbing region. Multiple choices of profile can be chosen for $\eta$ from linear to exponential.

### Defining the physical problem

The first step is to define the physical model:

* What are the physical dimensions of interest?
* What is the velocity profile of this physical domain?

We will use the velocity velocity model which we have just created using GemPy. As such we need to create a physical domain of the same size. As mentioned earlier, Devito and GemPy have slightly different grid implementations, which we will need to bear in mind to ensure our Devito model maps correctly to the GemPy grid.

We can now construct a Devito `Model`. This is a convenience object encapsulating the necessary parameters and components of an acoustic wave model, including additional damping layers around the perimeter (specified by `bcs="damp"`). For implementing custom damping setups, see `examples/userapi/04_boundary_conditions.ipynb` in the Devito repository. Note that we are using a relatively large number of damping layers here. This is to avoid our gathers becoming too messy, and ensure that reflections from horizons can be straightforwardly identified in the gathers. These damping layers are added to the edge of the specified model, increasing its extent (this is important for later visualisation).

In [ ]:
seis_model = Model(vp=reshaped, origin=(0., 0., -1000.), spacing=(10., 10., 10.), shape=shape, nbl=30, space_order=4, bcs="damp")

### Acquisition geometry
To fully define our problem setup we also need to define the source that injects the wave to model and the set of receiver locations at which to sample the wavefield. The source time signature will be modelled using a Ricker wavelet defined as

\begin{equation}
  q(t) = (1-2\pi^2 f_0^2 (t - \frac{1}{f_0})^2 )e^{- \pi^2 f_0^2 (t - \frac{1}{f_0})}
\end{equation}

To fully define the source signature we first need to define the time duration for our model and the timestep size, which is dictated by the CFL condition and our grid spacing. Luckily, our `Model` utility provides us with the critical timestep size, so we can fully discretize our model time axis as an array:

In [ ]:
from examples.seismic import TimeAxis

t0 = 0.  # Simulation starts at t=0
tn = 1000.  # Simulation last 1 second (1000 ms)
dt = seis_model.critical_dt  # Time step from model grid spacing

time_range = TimeAxis(start=t0, stop=tn, step=dt)

We will position our source at a depth of 20m, center it in all other axes, and set the peak wavelet frequency to 15Hz.

In [ ]:
from examples.seismic import RickerSource

f0 = 0.015  # Source peak frequency is 15Hz (0.015 kHz)
src = RickerSource(name='src', grid=seis_model.grid, f0=f0,
                   npoint=1, time_range=time_range)

# First, position source centrally in all dimensions, then set depth
src.coordinates.data[:] = np.array(seis_model.domain_size) * .5
src.coordinates.data[0, -1] = -20  # Depth is 20m

# We can plot the time signature to see the wavelet
src.show()

Similarly to our source object, we can now define our receiver geometry as a symbol of type `Receiver`. It is worth noting here that both utility classes, `RickerSource` and `Receiver` are thin wrappers around the Devito's `SparseTimeFunction` type, which encapsulates sparse point data and allows us to inject and interpolate values into and out of the computational grid. As we have already seen, both types provide a `.coordinates` property to define the position within the domain of all points encapsulated by that symbol. 

In this example we will position receivers at the same depth as the source, every $10m$ along the x axis, on the midline of the y axis. The `rec.data` property will be initialized, but left empty, as we will compute the receiver readings during the simulation.

In [ ]:
from examples.seismic import Receiver

# Create symbol for 101 receivers
rec = Receiver(name='rec', grid=seis_model.grid, npoint=101, time_range=time_range)

# Prescribe even spacing for receivers along the x-axis
rec.coordinates.data[:, 0] = np.linspace(0, seis_model.domain_size[0], num=101)
rec.coordinates.data[:, 1] = 0.5*seis_model.domain_size[1]
rec.coordinates.data[:, -1] = -20.  # Depth is 20m

### Wave equation and specification

As mentioned previously, we will be using the 2nd order acoustic wave equation in this tutorial. One of the key features of Devito is the ability to specify an equation, or system of equation, in a symbolic manner, and have this automatically discretized using the finite difference method.

In Devito, variables which vary in space only are represented using `Function` objects. If we also want them to vary over time, we must use a `TimeFunction`. There are further `Function`s for vector and tensor symbols: useful for concisely specifying the elastic wave equation for example. Each `Function` has a `.data` attribute, which contains the discretized values of the field. The FD discretization is specified using the `time_order` and `space_order` keyword arguments. In this case we will use a discretization with 2nd order accuracy in time, and 4th order accuracy in space.

We will create a function $u(t, x, y, z)$ to contain our pressure field. To do this, we need to provide a name (used when printing the function, or symbolic expressions which contain it), and a Devito `Grid` to which it should be attached (in this case, the grid included in the `Model` convenience object). Additional information about the discretization including time order, space order, and staggering can also be supplied.

In [ ]:
# Define the wavefield with the size of the model
u = dv.TimeFunction(name="u", grid=seis_model.grid, time_order=2, space_order=4)
u

We can also view the data attached to this `Function` as below. When the object is created, this is initialised to zero. 

In [ ]:
u.data.shape

There are also various convenient aliases contained within the `Function` object.

In [ ]:
u.shape == u.data.shape

Since we set `time_order=2` when creating the function, Devito has allocated three buffers to represent `u(t-1, x, y, z)`, `u(t, x, y, z)`, and `u(t+1, x, y, z)`, hence the size of the time dimension being 3. A larger buffer can be specified when creating the `TimeFunction` allowing all timesteps to be saved, but we have not done so in this case.

The `data` attribute is a Numpy array, and can be interacted with like any other.

In [ ]:
isinstance(u.data, np.ndarray)

To take the first of a variable `u` with respect to a dimension `x`, you can simply write `u.dx`, or `u.dx2` for the second derivative. For other dimensions (including time), simply swap out the `x`.

In [ ]:
# Show the derivative stencil
u.dx2.evaluate

### A detour into Finite Differences

This next section is included for completeness, explaining the derivation of finite difference stencils for given derivatives, and explicit timestepping schemes. Whilst an overview of the standard Taylor-series-based stencils is given, Devito also supports the specification of custom stencil coefficients necessary for various dispersion-relation preserving schemes. A tutorial for this can be found [here](https://github.com/devitocodes/devito/blob/master/examples/seismic/tutorials/07_DRP_schemes.ipynb). Note that there is also support for spatially-variant coefficients, either based on subdomains or over the full domain. 

To understand where this stencil comes from, lets have a quick overview of finite differences. Consider a function $f(x,t)$. Recall that the Taylor series of $f(x,t)$ in the spatial dimension takes the form

\begin{equation*}
 f(x+h,t)=f(x,t)+\frac{\partial f}{\partial x}h+\frac{1}{2}\frac{\partial^2 f}{\partial x^2}h^2+\frac{1}{3!}\frac{\partial^3 f}{\partial x^3}h^3+\frac{1}{4!}\frac{\partial^4 f}{\partial x^4}h^4+\ldots.
\end{equation*}

We can re-arrange the above expansion in the form

\begin{equation*}
 \frac{\partial f}{\partial x}=\frac{f(x+h,t)-f(x,t)}{h}-\frac{1}{h}\sum_{n=2}^{\infty}\frac{1}{n!}\frac{\partial^n f}{\partial x^n}h^n.
\end{equation*}

Thus, provided $h$ is small we can say

\begin{equation*}
 \frac{\partial f}{\partial x}\approx\frac{f(x+h,t)-f(x,t)}{h},
\end{equation*}

which will have an associated error

\begin{equation*}
-\frac{1}{h}\sum_{n=2}^{\infty}\frac{1}{n!}\frac{\partial^n f}{\partial x^n}h^n
\end{equation*}

In comparison to our result (again, provided $h$ is small) the associated error will therefore be proportional to $h^2$ (if $h$ is small, $h^2$ will be very small and $h^3$ will be tiny). It is common to write this as

\begin{equation*}
 \frac{\partial f}{\partial x}=\frac{f(x+h,t)-f(x,t)+\mathcal{O}(h^2)}{h}.
\end{equation*}

This is the well known *forward difference* approximation and is how spatial derivatives are approximated in 'space order' 1 finite difference schemes.

We can also write the following Taylor expansion

\begin{equation*}
 f(x-h,t)=f(x,t)-\frac{\partial f}{\partial x}h+\frac{1}{2}\frac{\partial^2 f}{\partial x^2}h^2-\frac{1}{3!}\frac{\partial^3 f}{\partial x^3}h^3+\frac{1}{4!}\frac{\partial^4 f}{\partial x^4}h^4+\ldots.
\end{equation*}

This leads to the backward difference approximation which is also 'first order accurate' (i.e. the error is proportional to $h^2$)

\begin{equation*}
 \frac{\partial f}{\partial x}\approx\frac{f(x,t)-f(x-h,t)}{h}.
\end{equation*}

Note now that if we look at $f(x+h)-f(x-h)$ and re-arrange we arrive at

\begin{equation*}
 \frac{\partial f}{\partial x}=\frac{f(x+h,t)-f(x-h,t)+\mathcal{O}(h^3)}{2h},
\end{equation*}

which is known as the centered difference approximation. Notice that now the $h^2$ terms cancel and the error is 'pushed' back to be of order $h^3$. That is, this approximation is (in theory) more accurate.

We can continue this process by considering more and more expansions and solving the resulting set of 'simultaneous equations' for $\partial f/\partial x$ to create higher and higher order approximations. For example, if we consider the expansions $f(x+2h,t)$ and $f(x-2h,t)$ we arrive at a 'fourth order' accurate scheme of the form

\begin{equation*}
 \frac{\partial f}{\partial x}=\frac{\frac{1}{12}f(x-2h,t)-\frac{2}{3}f(x-h,t)+\frac{2}{3}f(x+h,t)-\frac{1}{12}f(x+2h,t)+\mathcal{O}(h^5)}{h}.
\end{equation*}

Notice that as we increase the accuracy of our approximation the **stencil** becomes larger (and hence computations with higher order stencils become more demanding).

Through this process of manipulating Taylor series we can also derive approximations for other derivatives. For example, summing the series expansions $f(x+h,t)+f(x-h,t)$ and re-ordering we arrive at

\begin{equation*}
  \frac{\partial^2 f}{\partial x^2}=\frac{f(x-h,t)-2f(x,t)+f(x+h,t)+\mathcal{O}(h^3)}{h^2}.
\end{equation*}

Further, we can do the same thing for time derivatives (or other spatial dimensions in higher dimensional problems) and arrive at

\begin{equation*}
 \frac{\partial f}{\partial t}\approx\frac{f(x,t+\delta t)-f(x,t)}{\delta t},
\end{equation*}

where $\delta t$ is some small increment in time. This then allows us to create 'time-stepping schemes'. For example, consider the partial differential equation

\begin{equation*}
 \frac{\partial u}{\partial t}+c\frac{\partial u}{\partial x}=0,
\end{equation*}

discretizing only the temporal term for the time being we have

\begin{equation*}
 \frac{f(x,t+\delta t)-f(x,t)}{\delta t}+c\frac{\partial u}{\partial x}=0,
\end{equation*}

which we can re-arrange as

\begin{equation*}
 f(x,t+\delta t)=f(x,t)-\delta tc\frac{\partial u}{\partial x}.
\end{equation*}

which is one of the most basic *explicit* time stepping schemes. Now if we approximated our spatial derivative with a forward difference approximation our scheme will become

\begin{equation*}
 f(x,t+\delta t)=f(x,t)-\delta tc\frac{f(x+h,t)-f(x,t)}{h}.
\end{equation*}

Then, provided we know the status of our function $f$ at $t=0$ we can use the above scheme to compute the evolution of $f$ forward in time.

### Setting up the equation

With this out of the way, we can define our partial differential equation, essentially as we would write it on paper:

In [ ]:
# We can now write the PDE
pde = seis_model.m * u.dt2 - u.laplace + seis_model.damp * u.dt

# The PDE representation is as on paper
pde

Now we can create our update stencil to get the solution at the forward timestep. The value of `u` at the forward timestep can be accessed using `u.forward`.

In [ ]:
u.forward

Similarly, there is a `u.backward` for the backward timestep.

In [ ]:
u.backward

Additionally, derivatives can be taken at these positions using `u.forward.dx`.

In [ ]:
u.forward.dx

With this, we can solve our discretized PDE for u at the forward timestep, and set this equal to `u.forward`. This creates our update stencil.

In [ ]:
# This discrete PDE can be solved in a time-marching way updating u(t+dt) from the previous time step
# Devito as a shortcut for u(t+dt) which is u.forward. We can then rewrite the PDE as
# a time marching updating equation known as a stencil using customized SymPy functions

stencil = dv.Eq(u.forward, dv.solve(pde, u.forward))
stencil

## Break time! (Again)
Maybe have a biscuit this time?

### Source injection and receiver interpolation

With a numerical scheme to solve the homogenous wave equation, we need to add the source to introduce seismic waves and to implement the measurement operator, and interpolation operator. This operation is linked to the discrete scheme and needs to be done at the proper time step.

With this in mind, we can set up the source and reciever terms to include in our `Operator`.

In [ ]:
# Finally we define the source injection and receiver read function to generate the corresponding code
src_term = src.inject(field=u.forward, expr=src * dt**2 / seis_model.m)

# Create interpolation expression for receivers
rec_term = rec.interpolate(expr=u.forward)

### Devito operator and solve (where the magic happens)
After constructing all the necessary expressions for updating the wavefield, injecting the source term and interpolating onto the receiver points, we can now create the Devito operator that will generate the C code at runtime. When creating the operator, Devito performs multiple optimizations to reduce flop count and ensure memory locality. Devito can generate code to run on CPUs, GPUs, and clusters thereof, and makes use of several layers of parallelism.

**Note**: The argument `subs=model.spacing_map` causes the operator to substitute values for our current grid spacing into the expressions before code generation. This reduces the number of floating point operations executed by the kernel by pre-evaluating certain coefficients.

In [ ]:
op = dv.Operator([stencil] + src_term + rec_term, subs=seis_model.spacing_map)

Now we can execute the operator for a number of timesteps. We specify the number of timesteps to compute with the keyword `time` and the timestep size with `dt`.

In [ ]:
op(time=time_range.num-1, dt=seis_model.critical_dt)

### Plotting the synthetic shot record
We can now plot our shot record using everyone's favourite colourmap. We can clearly see the reflected arrivals from the seabed, top shale, and top CO2 (look for the reverse polarity arrival). We can also distinguish the base of the CO2 lens and the interface between the reservoir sandstone and the underlying shale.

In [ ]:
plt.imshow(rec.data, cmap='viridis', aspect='auto', vmax=0.01, vmin=-0.01, extent=(0, 100, tn, t0))
plt.xlabel("Reciever number")
plt.ylabel("Time (ms)")
plt.colorbar()
plt.show()

## Visualisation with PyVista:

In the final section of this tutorial, we will cover visualisation of the wavefield using PyVista: a dependency of GemPy. We can use its plotting and manipulation capabilities to visualise various slices of our wavefield.

Firstly, we need to import PyVista

In [ ]:
import pyvista as pv

With this done, we will want to take the 'data' attribute of 'u', containing the values of the field. This is a numpy array, indexed `[t, x, y, z]`, corresponding to the dimensions of `u`. As we want to plot the current (final) timestep, we will select time index 1.

In [ ]:
# Trim down the data from u to remove damping field
trimmed_data = u.data[1, 30:-30, 30:-30, 30:-30]

Now we will create a `UniformGrid`. This creates a simple uniform grid from a 3D NumPy array of values.

In [ ]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
grid.dimensions = np.array(trimmed_data.shape) + 1

# Edit the spatial reference
grid.origin = (0., 0., -1000.)  # The bottom left corner of the data set
grid.spacing = (10, 10, 10)  # These are the cell sizes along each axis

We can now fill the grid cells with our wavefield data:

In [ ]:
# Add the data values to the cell data
grid.cell_arrays["values"] = trimmed_data.flatten(order="F")  # Flatten the array!

And plot some orthogonal slices of this grid to visualise its internal structure:

In [ ]:
orth_slices = grid.slice_orthogonal(x=200, y=200, z=-500)

orth_slices.plot(cmap='seismic', clim=[-0.01, 0.01], notebook=True, window_size=(600,400))

An alternative visualisation is to take a series of slices along an axis. By setting each slice to be slightly transparent, it is possible to show a fairly complete image of the 3D structure without the need for in interactive plot.

In [ ]:
y_slices = grid.slice_along_axis(n=5, axis="y")
p = pv.Plotter(notebook=True, window_size=(600,400))
p.add_mesh(grid.outline(), color="k")
p.add_mesh(y_slices, cmap='seismic', clim=[-0.01, 0.01], opacity=0.8)
p.show()

## Further reading and exercise:

There is a further reading notebook `further_reading_fwi.ipynb` showing how to set up a toy full-waveform inversion problem using a slice of the velocity model we created in this tutorial for some inspiration on possible applications.

As an exercise, try adding an additional density parameter to the GemPy model, and use this to create a 1st-order (staggered) acoustic model with Devito.
Some tips:

* Look to `examples/seismic/tutorials/05_staggered_acoustic.ipynb` for a 1st-order acoustic example with constant physical parameters.
* Reshaping the data for use with Devito may be a little more involved. Make sure you separate the density and velocity data before applying the reshape.

## Thank you for listening!

The ecosystem of open-source geoscience software continues to grow, with high-quality packages like GemPy providing accessable, versatile tools to tackle all manner of scientific problems. In combination, we can use these to build new resources for the geoscience community, and ensure that they are adaptable and widely available. Devito is used commercially, and has been deployed at scale, allowing straightforward scaling between small examples and production-scale codes.

We look forward to seeing where you will take this: it would be great to see a 100% open-source, community-created Swung benchmark synthetic with Devito RTM/FWI examples by the time the next Transform rolls around. We envision generating synthetics for machine learning as one potential application, but are excited to see what applications the community will build from this tutorial.

I hope this tutorial has been useful. I will leave you with some helpful resources, and once again, feel free to ask on the Software Underground Slack or our own active Slack if you have any questions!

* [Devito website](https://www.devitoproject.org/)
* [Devito slack](https://devitocodes.slack.com/)
* [GemPy website](https://www.gempy.org/)

